In [2]:
# Welcome to the Fugitive Methane Containment Simulator, 
# developed by CC Lab intern Charlie Mayhew in summer 2021.

# The notebook below runs best through application/extension Voila — see
# https://voila.readthedocs.io/en/stable/using.html for information on installing.
# That said, it will work if you just execute each cell in order. 

# The IPython Widgets library I used is a little fussy, but I tried to make the code flexible enough
# for additional abatement options in the future.

# A few things are commented out (such as file saves), but this can easily be undone. — August 10, 2021.

# ~~~~~~~

# Import libraries.

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
%matplotlib inline
import pandas as pd
import scipy.integrate as integrate
import scipy.optimize as optimize
import fpdf
import smtplib, ssl

from ipywidgets import *
from IPython.display import display, Markdown, clear_output, Latex
from traitlets import traitlets
from matplotlib.backends.backend_pdf import PdfPages
from PyPDF2 import PdfFileMerger
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import date

# Some additional formatting choices. To alter font, change here and in the HTML cell below.

plt.style.use('ggplot')
style = {'description_width': '200px'}
layout = {'width':'500px'}
font = "Arial"
plt.rcParams.update({'font.family':font})
clear_output(wait=True)
filenames = []

# Only run the two lines below when demo'ing!

#import warnings
#warnings.filterwarnings("ignore")

# Decline curve coefficients from the EPA.

b_dict = {"vented":{"northern appalachian":{"low":2.023397, "mid":2.292595, "high":1.536968}, 
     "illinois":{"low":1.991397, "mid":2.314585, "high":1.656675},
     "central appalachian":{"low":1.960734, "mid":2.329011, "high":1.772918},
     "black warrior":{"low":1.947186, "mid":2.299685, "high":2.030229},
     "western states":{"low":1.957201, "mid":2.342465, "high":1.796142},
         "arb":{"mid":1.886581}}, 
          "sealed":{"northern appalachian":{"low":2.023397, "mid":2.292595, "high":1.536968}, 
     "illinois":{"low":1.991397, "mid":2.314585, "high":1.656675},
     "central appalachian":{"low":1.960734, "mid":2.329011, "high":1.772918},
     "black warrior":{"low":1.947186, "mid":2.299685, "high":2.030229},
     "western states":{"low":1.957201, "mid":2.342465, "high":1.796142},
         "arb":{"mid":2.016746}}}
D_dict = {"vented":{"northern appalachian":{"low":0.005831, "mid":0.003564, "high":0.001473},
       "illinois":{"low":0.005283, "mid":0.003564, "high":0.001611},
       "central appalachian":{"low":0.004790, "mid":0.003735, "high":0.001765},
       "black warrior":{"low":0.004734, "mid":0.003601, "high":0.002169},
       "western states":{"low":0.004734, "mid":0.003803, "high":0.001799},
         "arb":{"mid":0.003519}},
          "sealed":{"northern appalachian":{"low":0.001421, "mid":0.000725, "high":0.000487},
       "illinois":{"low":0.001315, "mid":0.000733, "high":0.000496},
       "central appalachian":{"low":0.001216, "mid":0.000741, "high":0.000506},
       "black warrior":{"low":0.001173, "mid":0.0007209, "high":0.000526},
       "western states":{"low":0.001205, "mid":0.000747, "high":0.000508},
         "arb":{"mid":0.000835}}}

status_scale = {'vented':1, 'sealed':0.5}
status = 'vented'

# Sample devices for testing. 

rto_1 = {'name': 'RTO 1', 'type': 'vam', 'start year':5, 'end year': 10, 'efficiency':0.97, 
         'operational hours':350*24, 'rto flow capacity': 158, 'vam concentration': 0.01,
         'capex':6, 'opex':1, 'haircut': 0}
    
flare_1 = {'name': 'Flare 1', 'type': 'amm', 'start year':10, 'end year': 15, 'efficiency':0.97, 'haircut':0,
         'operational hours':200, 'initial flow':1, 'initial flow units':'mcfd', 'capex':5, 'opex':0.3}

econ_1 = {'base credit price': 7, 'credit price escalator':0.01, 'opex escalator': 0.01}

sample_devices = [rto_1, flare_1]

In [3]:
%%html
<style>
body {
    font-family: "Arial";
}
</style>    

In [4]:
# First, define all underlying mathematic/economic functions.

# AMM decline curve emission rate.

def amm_q_rate(t, amm_q_i, start_year):
    amm_q = amm_q_i * status_scale[status] * ((1+b_dict[status]["arb"]["mid"]*365*D_dict[status]["arb"]["mid"]
                                       *(t-start_year)) ** (-1/b_dict[status]["arb"]["mid"]))
    return amm_q
                       
# Escalating credit price and OpEx.

def credit_price_func(t, economic_inputs):
    base_credit_price = economic_inputs['base credit price']
    credit_escalator = economic_inputs['credit price escalator']
    return base_credit_price * ((1+credit_escalator) ** (t))

def escalated_opex(t, base_opex, economic_inputs, start_year):
    opex_escalator = economic_inputs['opex escalator']
    return base_opex * ((1+opex_escalator) ** (t-start_year))

# Single device crediting and revenue rates. Right now penalties are added in by hand as a 'haircut.'

def single_device_credited_rate(device, t):
    start_year = device['start year']
    end_year = device['end year']
    efficiency = device['efficiency']
    operational_hours = device['operational hours']
    haircut = device['haircut']
    if device['type'] == 'vam':
        rto_flow_capacity = device['rto flow capacity']
        vam_concentration = device['vam concentration']
        if t < start_year:
            rto_burn_rate = 0
            penalties = 0
        if t >= start_year and end_year >= t:
            rto_burn_rate = (rto_flow_capacity * vam_concentration * efficiency * 60 * 0.483)
            penalties = 0 # add penalties
        if t > end_year:
            rto_burn_rate = 0
            penalties = 0
        annual_credits = (1 - haircut) * rto_burn_rate * operational_hours * (22.25/25) - penalties
        
    if device['type'] == 'amm':   
        amm_q_i = device['initial flow']
        if t < start_year:
            flare_rate = 0
            penalties = 0
        if t >= start_year and end_year >= t:
            
            flare_rate = amm_q_rate(t, amm_q_i, start_year) * efficiency * 0.483
                    # this is currently in MTCO2e/DAY     
            penalties = 0 # add penalties
        if t > end_year:
            flare_rate = 0
            penalties = 0
            
        annual_credits = (1 - haircut) * flare_rate * operational_hours / 24 * (22.25/25) - penalties  
    return annual_credits

def single_device_revenue_rate(device, t, economic_inputs):
    annual_revenue = (single_device_credited_rate(device, t) 
                      * credit_price_func(t, economic_inputs))
    return annual_revenue

# Costs.

def single_device_cost_rate(device, t, economic_inputs):
    start_year = device['start year']
    end_year = device['end year']
    capex = device['capex'] * 10 ** 6
    base_opex = device['opex'] * 10 ** 6
    year = int(t)
    if year < start_year-1:
        return 0
    if year == start_year-1:
        return - capex
    if year == start_year:
        return - escalated_opex(year, base_opex, economic_inputs, 0)
    if year > start_year and end_year >= year:
        return - escalated_opex(year, base_opex, economic_inputs, 0)
    if year > end_year:
        return 0
    
# Rates for all devices.
    
def all_devices_credited_rate(devices, t):
    return(np.sum([single_device_credited_rate(device, t) for device in devices]))

def cumulative_credits_func(devices, total_years):
    return integrate.quad(lambda t: all_devices_credited_rate(devices, t), 1, total_years)

def all_devices_revenue_rate(devices, t, economic_inputs):
    return(np.sum([single_device_revenue_rate(device, t, economic_inputs)
                  for device in devices]))

def all_devices_cost_rate(devices, t, economic_inputs):
    return(np.sum([single_device_cost_rate(device, t, economic_inputs) for device in devices]))

def cost_per_credit(devices, t, economic_inputs):
    min_start_year = min(device['start year'] for device in devices)
    if t < min_start_year:
        return 0
    else:
        year = int(t)
        credits_til_now = cumulative_credits_func(devices, year)[0]
        costs_til_now = - np.sum(np.array([all_devices_cost_rate(devices, 
                                                              x, economic_inputs) for x in range(0, year+1)]))
        return costs_til_now / credits_til_now

# Cashflows.

def single_device_cashflow(device, t, economic_inputs):
    return (single_device_revenue_rate(device, t, economic_inputs) 
            + single_device_cost_rate(device, t, economic_inputs))

def all_devices_cashflow(devices, t, economic_inputs):
    return np.sum([single_device_cashflow(device, t, economic_inputs)
                  for device in devices])

# IRR finding function first defines NPV as a function of IRR, then solves for the root of that function.
def irr_finder(devices, economic_inputs, total_years):
    int_years = np.linspace(0, total_years, num=total_years+1)
    annual_cashflow = [all_devices_cashflow(devices, t, economic_inputs) for t in int_years]
    def npv_func(irr):
        step1 = []
        for i in range(1, len(annual_cashflow)):
            step2 = annual_cashflow[i] / ((1 + irr) ** i)
            step1.append(step2)
        step3=np.array(step1)
        npv = annual_cashflow[0] + np.sum(step3)
        return npv
    try:
        sol = optimize.root_scalar(npv_func, x0=0.01, bracket=[-.99, 1], method='brentq')
        irr = round(sol.root * 100, 2)
        return(irr)
    except ValueError:
        pass

In [1]:
# Functions that 'process' inputs from interactive widgets into a structure the above functions can read.
# Note that each device is tracked in a dictionary, and that all devices are kept in an array of these
# dictionaries.

def collect_inputs_rto(name, concentration, vam_capex, vam_opex, flow_capacity, years, haircut):
    if name == "":
        name = "RTO"
        
    device = {'name': name, 'type': 'vam', 'start year':years[0], 'end year':years[1], 
                  'efficiency':0.97, 'operational hours':350 * 24, 'rto flow capacity': flow_capacity, 
                  'vam concentration': concentration * 0.01,'capex':vam_capex,'opex':vam_opex,
             'haircut': haircut * 0.01}
    display(Markdown("{} confirmed!".format(name)))
    devices_array.append(device)
    return  

def collect_inputs_amm(name, initial_flow, flow_units, amm_capex, amm_opex, years, haircut):
    if name == "":
        name = "Flare"
    if flow_units == 'MTCO2e/yr':
        initial_flow *= (2.07 / 365)
    device = {'name': name, 'type': 'amm', 'start year': years[0], 'end year': years[1],
             'efficiency':0.97, 'operational hours':300*24, 'initial flow': initial_flow,
             'capex': amm_capex, 'opex': amm_opex, 'haircut': haircut * 0.01}
    display(Markdown("{} confirmed!".format(name)))
    devices_array.append(device)
    return  

# The first interactive function, which is not the first interactive function presented to the user
# (that is defined farther below). After credit price, escalators, and overall project length have been
# defined, the user can add devices to their simulation.

def new_device_inputs(which_device, total_years):
    
    interact_named = interact_manual.options(manual_name='Confirm device')

    if which_device == 'RTO':
        name_wg = Text(placeholder='Name this RTO', description='Device Name',
                      disabled=False, style=style, layout=layout)
        concentration_wg = BoundedFloatText(value=0.5, min=0.3, max=1.2, step=0.01, 
                                                description='% CH4 Concentration', 
                                                disabled=False, style=style, layout=layout)
        vam_capex_wg = BoundedFloatText(value=5, min=0, max=15, step=0.01, description='VAM CapEx ($mm)',
                                            disabled=False, style=style, layout=layout)
        vam_opex_wg = BoundedFloatText(value=0.5, min=0, max=3, step=0.01, description='Base VAM OpEx ($mm)',
                                           disabled=False, style=style, layout=layout)
        flow_capacity_wg = BoundedFloatText(value=158, min=0, max=500, step=0.1, 
                                                description='RTO Flow Capacity (mcfm)', disabled=False, 
                                                style=style, layout=layout)
        years_wg = IntRangeSlider(value=[1, 10], min=1, max=total_years, 
                                  description='Years This RTO is Active', 
                                  disabled=False, style=style, layout=layout)
        haircut_wg = IntSlider(value=0, min=0, max=100, description='Penalties to Verified Credits (%)',
                                   disabled=False, style=style, layout=layout)
        device_attributes = interact_named(collect_inputs_rto, name=name_wg,concentration=concentration_wg, 
                                    vam_capex=vam_capex_wg, vam_opex=vam_opex_wg, 
                                    flow_capacity=flow_capacity_wg, years=years_wg, haircut=haircut_wg) 

    if which_device == 'Flare':
        name_wg = Text(placeholder='Name this flare', description='Device Name',
                      disabled=False, style=style, layout=layout)   
        amm_capex_wg = BoundedFloatText(value=1, min=0, max=15, step=0.01, description='AMM CapEx ($mm)',
                                            disabled=False, style=style, layout=layout)
        amm_opex_wg = BoundedFloatText(value=0.05, min=0, max=3, step=0.01, description='Base AMM OpEx ($mm)',
                                           disabled=False, style=style, layout=layout)
        years_wg = IntRangeSlider(value=[1, 10], min=1, max=total_years, 
                                  description='Years This Flare is Active', 
                                  disabled=False, style=style, layout=layout)
        initial_flow_wg = BoundedFloatText(value=500, min=0, max=10 ** 8, step=0.1, 
                                                description='Initial Flow Rate', disabled=False, 
                                                style=style, layout=layout)
        flow_units_wg = Dropdown(options=['mcfd', 'MTCO2e/yr'], description='Flow Rate Units',
                                disabled=False, style=style, layout=layout)
        haircut_wg = IntSlider(value=0, min=0, max=100, description='Penalties to Verified Credits (%)',
                                disabled=False, style=style, layout=layout)
        device_attributes = interact_named(collect_inputs_amm, name=name_wg, initial_flow=initial_flow_wg,
                                          flow_units= flow_units_wg, amm_capex=amm_capex_wg, 
                                           amm_opex=amm_opex_wg, years=years_wg, haircut=haircut_wg)

# Visualizations once all devices have been added: IRR, cashflows, cashflows per device, 
# and cost of production for 1 credit.
    
def visualizations(devices, economic_inputs_array, total_years_array):
    if len(devices) >= 1 and len(economic_inputs_array) >= 1:

        economic_inputs = economic_inputs_array[len(economic_inputs_array)-1]
        total_years = total_years_array[len(total_years_array)-1]
        project_lengths = range(0, total_years+1)        
        credit_price = economic_inputs["base credit price"]
        
        economic_inputs_low_cp = economic_inputs.copy()
        economic_inputs_low_cp["base credit price"] += -1
        economic_inputs_high_cp = economic_inputs.copy()
        economic_inputs_high_cp["base credit price"] += 1
        
        irrs = [irr_finder(devices, economic_inputs, t) for t in project_lengths]
        irrs_low_cp = [irr_finder(devices, economic_inputs_low_cp, t) for t in project_lengths]
        irrs_high_cp = [irr_finder(devices, economic_inputs_high_cp, t) for t in project_lengths] 
        
        cumulative_credits = round(cumulative_credits_func(devices, total_years)[0])
        
        cashflow = [all_devices_cashflow(devices, t, economic_inputs) for t in project_lengths]
        
        display(Markdown("Cumulative credits produced: {:,} MTCO2e".format(cumulative_credits)))
        display(Markdown("Total earnings: ${:,}".format(round(sum(cashflow)))))
        # display(Markdown("Final IRR: {}%".format(round(irrs[len(project_lengths)-1], 2))))
        
        # graphs = PdfPages('output-graphs.pdf')
        
        graph1 = plt.figure(figsize=(20, 6)) 
        graph1.tight_layout()
        irr_graph = plt.subplot(1, 2, 1)
        cashflow_graph = plt.subplot(1, 2, 2)
        irr_graph.plot(project_lengths, irrs, label = "Your IRR", color='blue')
        irr_graph.plot(project_lengths, irrs_low_cp, 
                       label = "IRR at base credit price = $%s" %str(credit_price-1),
                       linestyle='dashed', color='red')
        irr_graph.plot(project_lengths, irrs_high_cp, 
                       label = "IRR at base credit price = $%s" %str(credit_price+1),
                       linestyle='dashed', color='green')
        irr_graph.plot(project_lengths, [0 for x in project_lengths], 
                       color='black', linestyle='dashed', label='Break-even line')
        irr_graph.set_title("IRR vs. VAM Project Length")
        irr_graph.set_xlabel("Project Length (Years)")
        irr_graph.set_ylabel("IRR (%)")
        irr_graph.legend()
        
        cashflow_graph.bar(project_lengths, [c / (10**6) for c in cashflow], color='steelblue')
        cashflow_graph.set_xlabel("Years")
        cashflow_graph.set_ylabel("$mm")
        cashflow_graph.set_title("Annual Cash Flows for Inputed Scenario")
                
        # graphs.savefig(graph1)
        
        graph2 = plt.figure(figsize=(20, 6))
        graph2.tight_layout()
        per_credit = plt.subplot(1, 2, 1)
        
        min_start_year = int(min(device['start year'] for device in devices))
        per_credit_range = range(min_start_year+1, total_years+1)
        per_credit.plot(per_credit_range, [cost_per_credit(devices, t, economic_inputs) for t in
                                  per_credit_range], color='red', label='1 MTCO2e credit production cost')
        per_credit.plot(per_credit_range, [credit_price_func(t, economic_inputs) for t in
                                          per_credit_range], color='blue', 
                        linestyle='dashed', label='1 MTCO2e credit revenue')
        per_credit.legend()
        per_credit.set_xlabel("Years")
        per_credit.set_ylabel("$/MTCO2e")
        per_credit.set_title("Cost/Revenue for Single Credit across Project Lifetime")
        
        barWidth = 0.5
        number_of_ticks = np.arange(len(project_lengths))
        grouped_bar_heights = [[single_device_cashflow(device, t, economic_inputs) / (10 ** 6)
                                  for t in project_lengths] for device in devices]
       
        each_device = plt.subplot(1, 2, 2)
        for i in range(0, len(devices)):
            each_device.bar([x + i * barWidth for x in number_of_ticks], grouped_bar_heights[i],
                       width=barWidth, edgecolor='white', label=devices[i]["name"])
        each_device.set_xlabel("Year")
        each_device.set_ylabel("$mm")
        each_device.set_title("Cash Flows for Each Device")
        each_device.legend()
        
        # graphs.savefig(graph2)
        
        plt.show()
        graphs.close() 
        
    else:
        display(Markdown("Set your inputs!"))
        
def visualize(b):
    with output2:
        output2.clear_output(wait=True)
        visualizations(devices_array, economic_inputs_array, total_years_array)
        return

In [9]:
# Initialize arrays.

economic_inputs_array = []
total_years_array = []
devices_array = []

# Title of page.

display(Markdown("# Fugitive Methane Containment Simulator"))
display(Markdown("""<strong> Adjust the inputs below to build a methane abatement project 
                 – then see its emissions reductions, cash flows, and IRR."""))
print("""""")
display(Markdown("First, set overall project variables:"))

# Here are the initial widgets the user will see.

credit_price_wg = BoundedFloatText(value=7.5, min=1, max=20.0, step=0.01,description='Base Credit Price ($)',
                                disabled=False, style=style, layout=layout)
credit_price_esc_wg = FloatSlider(value=0, min=0, max=10, step=0.25, 
                                  description='Annual Credit Price Escalator (%)',
                                  disabled=False, style=style, layout=layout)
opex_esc_wg = FloatSlider(value=0, min=0, max=10, step=0.25, description='Annual OpEx Escalator (%)',
                       disabled=False, style=style, layout=layout)
total_years_wg = IntSlider(value=20, min=0, max=50, description='Total Project Years', 
                          disabled=False, style=style, layout=layout)

# After these initial values are confirmed, options to add devices appear. The reason why this happens 
# in stages is so the device widgets know what range of years they can be defined over.
    
def confirm_global_inputs(inputted_credit_price, inputted_credit_price_esc,
                             inputted_opex_esc, total_years):
    economic_inputs_array.append({'base credit price': inputted_credit_price , 
                           'credit price escalator': .01 * inputted_credit_price_esc,
                           'opex escalator': .01 * inputted_opex_esc})
    total_years_array.append(total_years)
    clear_output(wait=True)
    display(Markdown("Inputs saved!"))
    
    total_years_given = total_years_array[len(total_years_array)-1]
    
    def add_device(b):
        with output1:
            device_wg=Dropdown(options=['RTO', 
                                        'Flare', 'Gas well plug' 
                                       ],
                                 description='Device type: ', disabled=False)
    
            device_dropdown = interactive(new_device_inputs, which_device=device_wg, 
                                  total_years=fixed(total_years_given))
            display(device_dropdown)
        
    display(Markdown("Now, add your methane capture devices:"))
    device_button = Button(description='Add a new device')
    output1 = widgets.Output()
    device_button.on_click(add_device)
    display(device_button)
    display(output1)
    
    return

global_manual = interact_manual.options(manual_name='Save inputs')
out = global_manual(confirm_global_inputs, inputted_credit_price = credit_price_wg, 
                    inputted_credit_price_esc = credit_price_esc_wg, inputted_opex_esc = opex_esc_wg, 
                    total_years=total_years_wg)

simulation_button = Button(description='Simulate project')
output2 = widgets.Output()
simulation_button.on_click(visualize)
display(simulation_button, output2)

# Fugitive Methane Containment Simulator

<strong> Adjust the inputs below to build a methane abatement project 
                 – then see its emissions reductions, cash flows, and IRR.

First, set overall project variables:

interactive(children=(BoundedFloatText(value=7.5, description='Base Credit Price ($)', layout=Layout(width='50…

Button(description='Simulate project', style=ButtonStyle())

Output()